# Lecture 9
## Thursday, October 4th, 2018
### Automatic Differentiation:  Just Beyond the Basics

#### References
* [A Hitchhiker’s Guide to Automatic Differentiation](https://link.springer.com/article/10.1007/s11075-015-0067-6)
* Griewank, A. and Walther, A., 2008. Evaluating derivatives: principles and techniques of algorithmic differentiation (Vol. 105). Siam.
* Nocedal, J. and Wright, S., 2001. Numerical Optimization, Second Edition. Springer.

In [1]:
# Load libraries that we'll need
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
# or inline

# Recap
Last time:
* Provided introduction and motivation for automatic differentiation (AD)
  - Root-finding (Newton's method)
  - The Jacobian
  - The finite difference method
* The basics of AD
  - The computational graph
  - Evaluation trace
  - Computing derivatives of a function of one variable using the forward mode
  - Computing derivatives of a function of two variables using the forward mode

# Today
* The Jacobian in forward mode
* What the forward mode actually computes
* Dual numbers
* Towards implementation
* Thoughts on the reverse mode

---

# Finishing up the Basics

## Comments on Notation
A few students expressed concern with the "dot" notation from last time.  Admittedly, it is somewhat opaque notation.  However, it is also somewhat standard notation in the AD community (as much as any terminology is standard in the AD community).  You should feel free to use whatever notation makes you happy in this class.  Note, too, that when we start into the reverse mode, the itermediate steps are no longer denoted by a "dot".  In the reverse mode, intermediate steps are denoted with an "overbar".  This differing notation helps in distinguishing the two modes.

## Towards the Jacobian
Time to take a step back and recap.  In the very beginning of the lecture, we motivated calculation of derivatives by Newton's method for solving a system of nonlinear equations.  One of the key elements in that method is the Jacobian matrix $J = \dfrac{\partial f_{i}}{\partial x_{j}}$.  So far, you have learned how to manually perform the forward mode of automatic differentiation both for scalar functions of a single variable and scalar functions of multiple variables.  The solution of systems of equations requires differentation of a vector-function of multiple variables.

### Some Notation and the Seed Vector
Note that every time we computed the derivative using the forward mode, we "seeded" the derivative with a value.  We chose this value to be unity.  Let's look at a very simple function $f\left(x,y\right) = xy$.  Clearly $$\nabla f = \begin{bmatrix} y \\ x \end{bmatrix}.$$  The evaluation trace consists of $f = x_{3} = x_{1}x_{2}$ where $x_{1}$ and $x_{2}$ take on the values at the point we wish to evaluate the function and its derivative.  Let's introduce a seed vector $p$ and define the directional derivative to be $$D_{p}x_{3} = \sum_{j=1}^{2}{\dfrac{\partial x_{3}}{\partial x_{j}}p_{j}}.$$  Expanding the sum gives
\begin{align}
  D_{p}x_{3} &= \dfrac{\partial x_{3}}{\partial x_{1}}p_{1} + \dfrac{\partial x_{3}}{\partial x_{2}}p_{2} \\
             &= x_{2}p_{1} + x_{1}p_{2}.
\end{align}
Notice that if we choose the seed vector to be $p = \left(1,0\right)$ we get $\dfrac{\partial f}{\partial x}$ and if we choose $p = \left(0,1\right)$ we get $\dfrac{\partial f}{\partial y}$.  Of course, we aren't required to choose the seed vectors to be unit vectors.  However, the utility should be apparent.

Note that this could be written as $$D_{p}x_{3} = \nabla x_{3}\cdot p.$$  So, the forward mode of AD is really computing the *product* of the gradient of our function with the seed vector!

### What Forward AD Acutally Computes
The forward mode of automatic differentiation actually computes the product $\nabla f \cdot p$.  If $f$ is a vector, then the forward mode actually computes $Jp$ where $J = \dfrac{\partial f_{i}}{\partial x_{j}}, \quad i = 1,\ldots, n, \quad j = 1,\ldots,m$ is the Jacobian matrix.  In many applications, we really only want the "action" of the Jacobian on a vector.  That is, we just want to compute the matrix-vector product $Jp$ for some vector $p$.  Of course, if we really want to form the entire Jacobian, this is readily performed by forming a set of $m$ unit seed vectors $\left\{p_{1},\ldots p_{m}\right\}$ where $p_{k}$ consists of all zeros except for a single $1$ in position $k$.

### Exercise
That was a lot of jargon.  Let's do a little example to give the basic ideas.  Let
\begin{align}
  f\left(x,y\right) = 
  \begin{bmatrix}
    xy + \sin\left(x\right) \\
    x + y + \sin\left(xy\right)
  \end{bmatrix}.
\end{align}
The computational graph will have two inputs $x$ and $y$ and two outputs this time $x_{7}$ and $x_{8}$.  In the notation below $x_{7}$ corresponds to $f_{1}$ and $x_{8}$ corresponds to $f_{2}$.  Let's start by computing the gradient of $f_{1}$.  Using the directional derivative we have 
\begin{align}
  D_{p}x_{7} = \left(\cos\left(x_{1}\right) + x_{2}\right)p_{1} + x_{1}p_{2}.
\end{align}
**Note:** You should fill in the details!
Choosing $p = (1,0)$ gives $\dfrac{\partial f_{1}}{\partial x}$ and choosing $p = (0,1)$ gives $\dfrac{\partial f_{1}}{\partial y}$.  These form the first row of the Jacobian!  The second row of the Jacobian can be computed similarly by working with $x_{8}$.

The take-home message is that we can form the full Jacobian by using $m$ seed vectors where $m$ is the number of independent variables.

# Automatic Differentiation and Dual Numbers
A dual number is an extension of the real numbers.  Written out, the form looks similar to a complex number.

## Review of Comlex Numbers
Recall that a complex number has the form $$z = a + ib$$ where we *define* the number $i$ so that $i^{2} = -1$.  No real number has this property but it is a useful property for a number to have.  Hence the introduction of complex numbers.  Visually, you can think of a real number as a number lying on a straight line.  Then, we "extend" the real line "up".  The new axis is called the *imaginary* axis.

Complex numbers have several properties that we can use.
* Complex conjugate: $z^{*} = a - ib$.
* Magnitude of a complex number: $\left|z\right|^{2} = zz^{*} = \left(a+ib\right)\left(a-ib\right) = a^{2} + b^{2}$.
* Polar form: $z = \left|z\right|\exp\left(i\theta\right)$ where $\displaystyle \theta = \tan^{-1}\left(\dfrac{b}{a}\right)$.

## Towards Dual Numbers
A dual number has a real part and a dual part.  We write $$z = x + \epsilon x^{\prime}$$ and refer to $x^{\prime}$ as the dual part.  We *define* the number $\epsilon$ so that $\epsilon^{2} = 0$.  **This does not mean that $\epsilon$ is zero!**  $\epsilon$ is not a real number.

#### Some properties of dual numbers:
* Conjugate:  $z^{*} = x - \epsilon x^{\prime}$.
* Magnitude: $\left|z\right|^{2} = zz^{*} = \left(x+\epsilon x^{\prime}\right)\left(x-\epsilon x^{\prime}\right) = x^{2}$.
* Polar form: $z = x\left(1 + \dfrac{x^{\prime}}{x}\right)$.

### Example
Recall that the derivative of $y=x^{2}$ is $y^{\prime} = 2xx^{\prime} = 2x$.

Now if we extend $x$ so that it has a real part and a dual part ($x\leftarrow x + \epsilon x^{\prime}$) and evaluate $y$ we have
\begin{align}
  y &= \left(x + \epsilon x^{\prime}\right)^{2} \\
    &= x^{2} + 2xx^{\prime}\epsilon + \underbrace{x^{\prime^{2}}\epsilon^{2}}_{=0} \\
    &= x^{2} + 2xx^{\prime}\epsilon.
\end{align}
#### Notice that the dual part contains the derivative of our function!!

### Example
Evaluate $y = \sin\left(x\right)$ when $x\leftarrow x + \epsilon x^{\prime}$.

We have
\begin{align}
  y & = \sin\left(x + \epsilon x^{\prime}\right) \\
    & = \sin\left(x\right)\cos\left(\epsilon x^{\prime}\right) + \cos\left(x\right)\sin\left(\epsilon x^{\prime}\right).
\end{align}
Expanding $\cos$ and $\sin$ in their Taylor series gives 
\begin{align}
  \sin\left(\epsilon x^{\prime}\right) &= \sum_{n=0}^{\infty}{\left(-1\right)^{n}\dfrac{\left(\epsilon x^{\prime}\right)^{2n+1}}{\left(2n+1\right)!}} = \epsilon x^{\prime} + \dfrac{\left(\epsilon x^{\prime}\right)^{3}}{3!} + \cdots = \epsilon x^{\prime} \\
  \cos\left(\epsilon x^{\prime}\right) &= \sum_{n=0}^{\infty}{\left(-1\right)^{n}\dfrac{\left(\epsilon x^{\prime}\right)^{2n}}{\left(2n\right)!}} = 1 + \dfrac{\left(\epsilon x^{\prime}\right)^{2}}{2} + \cdots = 1.
\end{align}
Note that the definition of $\epsilon$ was used which resulted in the collapsed sum.

So we see that 
\begin{align}
  y & = \sin\left(x\right) + \cos\left(x\right) x^{\prime} \epsilon.
\end{align}
And once again the real component is the function and the dual component is the derivative.

### Exercise
Using dual numbers, find the derivative of $$ y = e^{x^{2}}.$$  **Show your work!**

# Thoughts on Implementation

There are different ways of implementing automatic differentiation.  Two predominant approaches are code translation through pre-processor directives and operator-overloading.  We won't much to say about code translation.  It can be quite complicated, even going so far as to write a compiler.  The high-level idea is that the compiler writes additional code to compute derivatives of functions.  The benefits are that it can be very efficient and fast.

Our approach will use operator overloading.

## Operator Overloading
The Wikipedia article on [operator overloading](https://en.wikipedia.org/wiki/Operator_overloading) states: 
> ...operator overloading, sometimes termed operator ad hoc polymorphism, is a specific case of polymorphism, where different operators have different implementations depending on their arguments.

You have already seen the basics of this although we didn't call it operator overloading.  `python` allows operator overloading via the special methods.  When you wrote the circle class, some of you tried to define the `__add__` method (and `__radd__` for communtivity) to add two circle objects.  In this way, you *overloaded* the additon operator (`+`).

### Example
Let's create a class to do complex numbers again.

In [2]:
# Complex class again
class Complex():
    def __init__(self, a, b):
        # Constructor to set up real and imag parts
        self.a = a
        self.b = b

# Create some complex numbers
z1 = Complex(0, 1)
z2 = Complex(2, 0)

# And add them
z = z1 + z2
print(z.a, z.b)

TypeError: unsupported operand type(s) for +: 'Complex' and 'Complex'

That didn't work.  `python` doesn't know how to apply the addition operator (`+`) to `Complex` objects!  Let's remedy that.

In [3]:
# Complex class again
class Complex():
    def __init__(self, a, b):
        # Constructor to st up real and imag parts
        self.a = a
        self.b = b
    
    def __add__(self, other):
        # Implementating complex addition
        return Complex(self.a + other.a, self.b + other.b)

z1 = Complex(0, 1)
z2 = Complex(2, 0)
z = z1 + z2
print(z.a, z.b)

2 1


Okay, much better.  Of course, there are still many problems here:
* No convenient way to print out a complex number (where's `__str__`?)
* What happens when you try to add a complex number to a real number (e.g. `2 + z`)?
  - Try it!
  - Then check out `__radd__`.
  - This is how you get commutivity.
* Many operations are missing (e.g. where's multiplication?).

Nevertheless, the addition operator has been (partly) successfully overloaded (we didn't get commutivity yet).

## Implementation with the Duals
This section will give you a hint on how you may go about implementing the forward mode.  I will be deliberately terse.  In fact, I will not even define a class or use any operator overloading.  What I'm about to show you is almost purely conceptual.  You will do the hard work of coming up with a user-friendly implementation for your project.

Let's start with a baby example.  Suppose at some point in our evaluate trace we have $$x_{3} = x_{1}x_{2}.$$  We want to evaluate $x_{3}$ and the derivative.  Following our usual conventions we have $$\dot{x}_{3} = x_{1}\dot{x}_{2} + \dot{x}_{1}x_{2}.$$  For our purposes here, suppose we have that $x_{1} = 2$ and $x_{2} = 3$ as well as $\dot{x}_{1} = 1$ and $\dot{x}_{2} = 4$.  It doesn't matter where these values came from at the moment.

Let's define an object that has two components.  The first component will hold the value of the function and the second component will hold the value of the derivative.

In [4]:
#     x1   x1 dot
x1 = [2,     1] # value and derivative
x2 = [3,     4]

It's clear how to compute the value of the function at the point $a$.  Simply use the first component of our object to compute the function value.

In [5]:
f = [0, 0] # Allocate the values of the function and its derivative
# f(a) = x1    * x2
f[0]   = x1[0] * x2[0]
print(f)

[6, 0]


But to calculate the derivative, we need to *overload* multiplication.  Any time we encounter a multiplication of something in the second slot, we need to use the chain rule.

In [6]:
#f'(a) = x1    * x2dot + x1dot * x2
f[1]   = x1[0] * x2[1] + x1[1] * x2[0]
print(f)

[6, 11]


Let's recap.  We'll use OOP vernacular.
* We instantiated two gradient objects $x_{1}$ and $x_{2}$ (here just lists).  These objects have two name attributes.  The first attribute is the function value and the second attribute is the derivative.  When performing a multiplication, the usual multiplication operator can be used for the function value part of the object.  However, for the derivative part, one must overload the multiplication operator.  Note that this could be accomplished in a class by implementing the `__mul__` and `__rmul__` special methods and returning the gradient object (like we did for `__add__` in the `Complex` class example).

### Exercise
Your turn.  Using the same approach as above, write some code to do forward mode automatic differentiation of the function $$f\left(x\right) = x^{r}$$ where $r \in \mathbb{R}$.  Evaluate the derivative and function at $a = 3$ and $r = 4$.

You can turn this into a function or closure if that helps you with the concepts.

# Comments on the Reverse Mode
The focus of this class is on the forward mode of automatic differentiation.  The reverse mode is also extremely popular and useful (e.g. in scenarios that have $f:\mathbb{R}^{m}\mapsto\mathbb{R}$).  Here we will outline the mechanics of the reverse mode, show a little example, and survey the basic equations.

## A Sketch of the Reverse Mode
* Create evaluation graph
* Forward pass does function evaluations
* Forward pass also does partial derivatives of elementary functions
  * **It does NOT do the chain rule!**
  * Just stores the partial derivatives
  * If $x_{3} = x_{1}x_{2}$ is a node, we store $\dfrac{\partial x_{3}}{\partial x_{1}}$ and $\dfrac{\partial x_{3}}{\partial x_{2}}$.  That's it.
* Reverse pass starts with $\overline{x}_{N} = \dfrac{\partial f}{\partial x_{N}} = 1$ (since $f$ is $x_{N}$)
* Then it gets $\overline{x}_{N-1} = \dfrac{\partial f}{\partial x_{N}}\dfrac{\partial x_{N}}{\partial x_{N-1}}$
  * **Note:** $\dfrac{\partial x_{N}}{\partial x_{N-1}}$ is already stored from the forward pass
* The only trick occurrs when we get to a branch in the graph.  That is, when the node we're on has more than one child.  In that case, we sum the two paths.  For example, if $x_{3}$ has $x_{4}$ and $x_{5}$ as children, then we do $$\overline{x}_{3} = \dfrac{\partial f}{\partial x_{3}} = \dfrac{\partial f}{\partial x_{4}}\dfrac{\partial x_{4}}{\partial x_{3}} + \dfrac{\partial f}{\partial x_{5}}\dfrac{\partial x_{5}}{\partial x_{3}}.$$
  * **Note:** This summation is a manifestation of the chain rule.


The reverse mode cannot be interpreted in the context of dual numbers like the forward mode was.  The little implementation sketch that we did for the forward mode will need to be generalized for the reverse mode.

## The Basic Equations
These equations are modified from Nocedal and Wright (page 180).

The partial derivative of $f$ with respect to $x_{i}$ can be written as 
\begin{align}
  \dfrac{\partial f}{\partial x_{i}} = \sum_{\text{j a child of i}}{\dfrac{\partial f}{\partial x_{j}}\dfrac{\partial x_{j}}{\partial x_{i}}}.
\end{align}
At each node $i$ we compute 
\begin{align}
  \overline{x}_{i} += \dfrac{\partial f}{\partial x_{j}}\dfrac{\partial x_{j}}{\partial x_{i}}.
\end{align}
The $\overline{x}_{i}$ variable stores the current value of the partial derivative at node $i$.  It is sometimes called the adjoint variable.

## An Example for Intuition
Let's try to evaluate the function $$f\left(x,y\right) = xy + \exp\left(xy\right)$$ and its gradient at the point $a = (1,2)$.  We'll use the reverse mode this time.

Clearly we have $$\nabla f = \begin{bmatrix} y + \exp\left(xy\right)y \\ x + \exp\left(xy\right)x \end{bmatrix}.$$  Hence
\begin{align}
  f\left(a\right) &= 2 + e^{2} \\
  \nabla f &= 
  \begin{bmatrix} 
    2 + 2e^{2} \\ 
    1 + e^{2} 
  \end{bmatrix}.
\end{align}

Here's a visualization of the computational graph:

![](figs/Reverse-Example.png)

Let's use the reverse mode to calculate the gradient of $f$.

1. Generate the forward trace and calculate the partial derivatives of a node wrt its children
  * Notice that this time we need to save the graph
2. Start at $x_{5}$ and start calculating the chain rule